In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-04-29"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
8234,2024-04-29,Eua Nba,20:30,Miami Heat,Boston Celtics,5.04,1.19,202.5,1.80,1.95,9.5,2.00,1.03,xtdtbE3e,0.198413,0.840336,0.555556,0.512821,0.038749,0.0,0.4,NaN,NaN,219.584,18.455680,0.084048,172.090,17.189677,0.099888,222.664,167.714,202.72,161.98,0.0,0.0,0.0,0.0,0.873952,0.056569,0.452735,-1.87,-0.374,-10.802139,0.496817,0.4,-0.096817,-0.91,-0.182,-1.043956,0.689363,0.8,0.110637
8235,2024-04-29,Eua Nba,23:00,Denver Nuggets,Los Angeles Lakers,1.31,3.67,216.5,1.80,1.95,-7.5,1.93,7.50,nRBwQuio,0.763359,0.272480,0.555556,0.512821,0.035838,0.4,0.2,NaN,NaN,178.210,23.225684,0.130328,253.726,65.079154,0.256494,154.926,231.356,152.76,353.29,0.0,0.0,0.0,0.0,0.670190,0.056569,0.835331,0.56,0.112,2.767857,0.753165,0.9,0.146835,-1.44,-0.288,-9.270833,0.493305,0.5,0.006695
8236,2024-04-29,Dinamarca Liga De Basquete,14:15,Horsens,Næstved,1.76,2.00,174.5,1.80,1.86,-2.5,2.02,2.38,tfUYSSr2,0.568182,0.500000,0.555556,0.537634,0.068182,0.0,0.6,NaN,NaN,100.204,27.804581,0.277480,205.906,86.468250,0.419940,0.000,154.626,88.58,194.93,0.0,0.0,1.0,0.0,0.090269,0.023184,0.115708,0.00,0.000,inf,0.000000,0.0,0.000000,0.95,0.190,5.263158,0.000000,0.0,0.000000
8237,2024-04-29,Finlândia Korisliiga,12:30,Kataja,BC Nokia,1.77,1.99,163.5,1.85,1.81,-2.5,2.01,2.39,2Zyby9LF,0.564972,0.502513,0.540541,0.552486,0.067484,0.6,0.4,NaN,NaN,202.926,94.975633,0.468031,149.248,20.583915,0.137918,206.774,145.206,148.33,168.35,0.0,0.0,0.0,0.0,0.082747,0.015456,0.122137,4.68,0.936,0.822650,0.000000,0.0,0.000000,-0.22,-0.044,-22.500000,0.000000,0.0,0.000000
8238,2024-04-29,Irã Superliga,09:30,Zob Ahan,Tabiat,1.74,1.99,156.5,1.85,1.85,-2.5,2.01,2.40,QkQKEYYO,0.574713,0.502513,0.540541,0.540541,0.077225,0.2,0.6,NaN,NaN,134.326,36.018925,0.268146,130.080,30.591845,0.235177,129.936,135.114,127.28,169.68,0.0,0.0,0.0,0.0,0.094786,0.000000,0.125067,-1.01,-0.202,-3.663366,0.000000,0.0,0.000000,-0.22,-0.044,-22.500000,0.000000,0.0,0.000000
8239,2024-04-29,Letônia Lbl,13:30,Rigas Zelli,Ventspils,1.75,2.01,169.5,1.87,1.79,-2.5,2.01,2.40,G2aRLnyG,0.571429,0.497512,0.534759,0.558659,0.068941,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.097791,0.030912,0.125067,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
8240,2024-04-29,Libano Divisão 1,15:45,Beirut Club,Sagesse Al Hekmeh Beirut,1.70,2.05,172.5,1.85,1.85,-2.5,1.95,2.49,z53VF9Lk,0.588235,0.487805,0.540541,0.540541,0.076040,1.0,0.4,NaN,NaN,123.046,39.031603,0.317211,175.026,139.135149,0.794940,111.142,163.156,117.42,421.60,0.0,0.0,0.0,0.0,0.131993,0.000000,0.171999,-0.69,-0.138,-5.072464,0.000000,0.0,0.000000,-1.41,-0.282,-3.723404,0.000000,0.0,0.000000
8241,2024-04-29,Romênia Divisão A,13:00,Dinamo Bucharest,Rapid Bucuresti,1.69,2.06,151.5,1.85,1.85,-2.5,1.94,2.50,4zQPYf3L,0.591716,0.485437,0.540541,0.540541,0.077153,0.6,0.6,NaN,NaN,165.572,42.177543,0.254738,147.524,35.095821,0.237899,161.950,157.358,170.52,134.30,0.0,0.0,0.0,0.0,0.139536,0.000000,0.178369,0.85,0.170,4.058824,0.000000,0.0,0.000000,0.27,0.054,19.629630,0.000000,0.0,0.000000
8242,2024-04-29,Eua Nba,21:30,New Orleans Pelicans,Oklahoma City Thunder,2.61,1.53,205.5,1.93,1.93,3.5,2.05,1.30,QaNULxDa,0.383142,0.653595,0.518135,0.518135,0.036737,0.2,0.6,NaN,NaN,215.370,38.333001,0.177987,226.066,40.111109,0.177431,210.424,225.462,167.45,200.34,0.0,0.0,0.0,0.0,0.368925,0.000000,0.316615,-2.98,-0.596,-2.701342,0.551633,0.4,-0.151633,0.70,0.140,

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,23:00,Eua Nba,Denver Nuggets,Los Angeles Lakers,216.5,1.80,1.0,Over
1,14:15,Dinamarca Liga De Basquete,Horsens,Næstved,174.5,1.80,1.0,Over
2,12:30,Finlândia Korisliiga,Kataja,BC Nokia,163.5,1.85,1.0,Over
3,09:30,Irã Superliga,Zob Ahan,Tabiat,156.5,1.85,1.0,Over
4,13:30,Letônia Lbl,Rigas Zelli,Ventspils,169.5,1.87,1.0,Over
5,15:45,Libano Divisão 1,Beirut Club,Sagesse Al Hekmeh Beirut,172.5,1.85,1.0,Over
6,13:00,Romênia Divisão A,Dinamo Bucharest,Rapid Bucuresti,151.5,1.85,1.0,Over
